In [4]:
import pandas as pd
import numpy
import torch 
import numpy as np
from torch.utils.data import Dataset, DataLoader
import sys
sys.path.append("..")
from utils import pad_collate
from dataloader_comma import CommaDataset
from dataloader_nuscenes import NUScenesDataset
from model import VTN
import matplotlib.pyplot as plt 
from PIL import Image
import glob
import os
from utils import * 
import re
from pathlib import Path
import argparse
import yaml

In [5]:
def rmse_loss(input, target, mask, reduction="mean"):
        out = (input[~mask]-target[~mask])**2
        return out.mean().sqrt() if reduction == "mean" else out 

def mae_loss(input, target, mask, reduction="mean"):
        out = (input[~mask]-target[~mask])
        return out.mean().abs() if reduction == "mean" else out 

In [6]:
def argparse_namespace_constructor(loader, node):
    values = loader.construct_mapping(node)
    namespace = argparse.Namespace()
    namespace.__dict__.update(values)
    return namespace

# Register the custom constructor with PyYAML
yaml.add_constructor('tag:yaml.org,2002:python/object:argparse.Namespace', argparse_namespace_constructor)


In [7]:
elems = []
n_feats = []
p = '/data1/jessica/data/toyota/ckpts_final/'
for elem in os.listdir(p):
    l = p + elem + "/lightning_logs"
    if len(os.listdir(p + elem)) == 0: continue
    for version in os.listdir(l):
        k = f'{l}/{version}/checkpoints/'
        if 'checkpoints' not in os.listdir(f'{l}/{version}'): continue 
        for filename in os.listdir(k):
            if "yaml" in filename:
                with open(k + filename, "r") as file:
                    yaml_data = yaml.load(file, Loader=yaml.FullLoader)
                    data_dict = vars(yaml_data)
                if 'n_scenarios' in data_dict:
                    print(k + filename)
                    elems.append(k)
                    n_feats.append(data_dict['n_scenarios'])


/data1/jessica/data/toyota/ckpts_final/ckpts_final_comma_distance_none_True/lightning_logs/version_4/checkpoints/hparams.yaml
/data1/jessica/data/toyota/ckpts_final/ckpts_final_comma_distance_none_True/lightning_logs/version_6/checkpoints/hparams.yaml
/data1/jessica/data/toyota/ckpts_final/ckpts_final_comma_distance_none_True/lightning_logs/version_2/checkpoints/hparams.yaml
/data1/jessica/data/toyota/ckpts_final/ckpts_final_comma_distance_none_True/lightning_logs/version_5/checkpoints/hparams.yaml
/data1/jessica/data/toyota/ckpts_final/ckpts_final_nuscenes_multitask_none_True/lightning_logs/version_3/checkpoints/hparams.yaml
/data1/jessica/data/toyota/ckpts_final/ckpts_final_nuscenes_multitask_none_True/lightning_logs/version_2/checkpoints/hparams.yaml
/data1/jessica/data/toyota/ckpts_final/ckpts_final_nuscenes_multitask_none_True/lightning_logs/version_5/checkpoints/hparams.yaml
/data1/jessica/data/toyota/ckpts_final/ckpts_final_nuscenes_distance_none_True_0.5/lightning_logs/version_

In [8]:
res = {}
for elem in elems:
    task = []
    for filename in os.listdir(elem):
        if '.csv' in filename: 
            df = pd.read_csv(elem + filename)
            if "nuscenes" in filename:
                df = df[0:1400]
            df.columns = ['preds', 'targets']
            m = (df['targets'] == 0).astype(bool) | (df['targets'] > 50).astype(bool)  if "angle" not in elem  else (df['targets'] == np.inf).astype(bool)
            
            
            loss3 = mae_loss(torch.tensor(df['preds']),  torch.tensor(df['targets']), torch.tensor(m))
            task.append(round(loss3.item(), 2))
            res[filename] = task


In [21]:

rows = []
for elem in res.keys():
    task = "dist" if "dist" in elem else ("angle" if 'angle' in elem else "multitask")
    dataset = "comma" if 'comma' in elem else "nuscenes"
    size = 100 if '100' in elem else (300 if '300' in elem else (24 if '24' in elem else 48 if '48' in elem else "none"))
    val = res[elem]
    if task == 'multitask' or size == "none" or dataset != "nuscenes": continue
    rows.append({'task': task, 'dataset': dataset, "MAE": val[0], "size":size})

In [22]:
pd.DataFrame(rows).sort_values('size').sort_values('task')

,task,dataset,MAE,size
3,angle,nuscenes,1.49,24
5,angle,nuscenes,0.60,48
4,angle,nuscenes,4.01,300
0,dist,nuscenes,1.85,24
1,dist,nuscenes,0.02,48
2,dist,nuscenes,2.27,300


In [6]:

p = '/data1/jessica/data/toyota/ckpts_final/'

experiments = os.listdir(p)
res = {}
res_abl = {}
res_ckpt = {}
for elem in experiments:
    path = p + elem + "/lightning_logs/" 
    vs = os.listdir(path)
    filt = []
    for elem1 in vs: 
        if 'version' in elem1:
            filt.append(elem1)
    versions =[int(elem.split("_")[-1])for elem in filt]
    versions = sorted(versions)
    if len(versions) == 0: continue
    version = f"version_{versions[-1]}"
    print(versions, version)
    #if elem == 'ckpts_final_comma_distance_clip' or elem == 'ckpts_final_comma_multitask_vit': version = "version_0"
    

    checkpoint_path = path + version + "/checkpoints/"
    if "checkpoints" not in os.listdir(path + version): continue
    files = os.listdir(checkpoint_path)

    task = []
    ckpt = []
    task_abl = []
    files = sorted(files)
    for filename in files: 
        
        if filename.endswith(".csv"):
            df = pd.read_csv(checkpoint_path + filename)
            if "nuscenes" in filename:
                df = df[0:1400]
            df.columns = ['preds', 'targets']
            m = (df['targets'] == 0).astype(bool) | (df['targets'] > 70).astype(bool)  if "angle" not in elem  else (df['targets'] == np.inf).astype(bool)
            
            loss3 = mae_loss(torch.tensor(df['preds']),  torch.tensor(df['targets']), torch.tensor(m))
            if "00" in filename:
                task_abl.append(round(loss3.item(), 2))
            else:
                task.append(round(loss3.item(), 2))
        if filename.endswith(".ckpt"):
            ckpt.append(checkpoint_path + '/' + filename)
    res[elem] = task
    if len(task_abl) != 0:
        res_abl[elem] = task_abl
    res_ckpt[elem] = ckpt
rows = []
for elem in res.keys():
    splitted = elem.split("_")
    data = splitted[2]
    task = splitted[3]
    backbone = splitted[4]
    if len(splitted) > 5:
        concept = splitted[5]
    else: 
        concept = False
    rows.append({"Concept": concept, "Task": task, "Backbone": backbone, "Dataset": data, "t-RMSE": round(res[elem][0], 2) if len(res[elem]) == 1 else res[elem]})




[0] version_0


NameError: name 'mae_loss' is not defined

In [4]:
res_abl

{'ckpts_final_comma_angle_none_True': [0.27]}

In [5]:
rows

[{'Concept': False,
  'Task': 'distance',
  'Backbone': 'resnet',
  'Dataset': 'comma',
  't-RMSE': 2.9},
 {'Concept': False,
  'Task': 'multitask',
  'Backbone': 'none',
  'Dataset': 'comma',
  't-RMSE': [0.4, 1.15]},
 {'Concept': 'True',
  'Task': 'angle',
  'Backbone': 'resnet',
  'Dataset': 'comma',
  't-RMSE': 0.37},
 {'Concept': False,
  'Task': 'angle',
  'Backbone': 'vit',
  'Dataset': 'nuscenes',
  't-RMSE': 3.75},
 {'Concept': 'True',
  'Task': 'distance',
  'Backbone': 'none',
  'Dataset': 'comma',
  't-RMSE': []},
 {'Concept': False,
  'Task': 'multitask',
  'Backbone': 'none',
  'Dataset': 'nuscenes',
  't-RMSE': [0.36, 6.65]},
 {'Concept': False,
  'Task': 'multitask',
  'Backbone': 'resnet',
  'Dataset': 'nuscenes',
  't-RMSE': [9.42, 43.81]},
 {'Concept': 'True',
  'Task': 'multitask',
  'Backbone': 'none',
  'Dataset': 'nuscenes',
  't-RMSE': []},
 {'Concept': False,
  'Task': 'multitask',
  'Backbone': 'resnet',
  'Dataset': 'comma',
  't-RMSE': [0.39, 3.48]},
 {'Conc

In [6]:
# The list of dictionaries
df = pd.DataFrame(rows)

# Pivot the DataFrame
df_pivot = pd.pivot_table(df, values='t-RMSE', index=['Dataset', 'Backbone', 'Concept'], columns='Task', aggfunc=lambda x: x)

# Rename the columns
df_pivot.columns = [f'{col}-MAE' for col in df_pivot.columns]
# Reset the index
df_pivot = df_pivot.reset_index()
#df_pivot['Feat. Size'] = [512, 643, 512, 768, 512, 643, 512, 768]
#df_pivot = df_pivot[["Dataset", "Backbone", 'Feat. Size', 'angle-MAE', 'distance-MAE', 'multiangle-MAE', 'multidistance-MAE']].round(2)



In [7]:
df_pivot

,Dataset,Backbone,Concept,angle-MAE,distance-MAE,multitask-MAE
0,comma,clip,False,0.03,4.05,"[0.24, 4.67]"
1,comma,none,False,0.7,1.85,"[0.4, 1.15]"
2,comma,none,True,[],[],[]
3,comma,resnet,False,0.03,2.9,"[0.39, 3.48]"
4,comma,resnet,True,0.37,[],"[2.15, 1.74]"
5,comma,vit,False,0.06,0.1,"[0.81, 3.13]"
6,nuscenes,clip,False,1.8,5.46,"[3.55, 3.47]"
7,nuscenes,none,False,1.95,4.21,"[0.36, 6.65]"
8,nuscenes,none,True,[],[],[]
9,nuscenes,resnet,False,5.87,26.5,"[9.42, 43.81]"


In [17]:
df

,Concept,Task,Backbone,Dataset,t-RMSE
0,False,distance,resnet,comma,4.24
1,False,multitask,none,comma,"[0.03, 1.87]"
2,True,angle,resnet,comma,0.84
3,False,angle,vit,nuscenes,3.75
4,False,multitask,none,nuscenes,"[0.36, 6.65]"
5,False,multitask,resnet,nuscenes,"[9.42, 43.81]"
6,False,multitask,resnet,comma,"[0.25, 3.85]"
7,False,angle,resnet,comma,0.32
8,False,multitask,vit,nuscenes,"[0.96, 16.62]"
9,True,multitask,resnet,comma,"[2.08, 1.84]"


In [18]:
df_pivot = df_pivot[["Dataset", "Backbone", 'angle-MAE', 'distance-MAE', 'multitask-MAE']].round(2)
df_pivot

,Dataset,Backbone,angle-MAE,distance-MAE,multitask-MAE
0,comma,clip,0.02,3.54,"[0.18, 4.11]"
1,comma,none,0.29,2.46,"[0.03, 1.87]"
2,comma,resnet,0.32,4.24,"[0.25, 3.85]"
3,comma,resnet,0.84,[],"[2.08, 1.84]"
4,comma,vit,0.01,0.85,"[1.05, 4.41]"
5,nuscenes,clip,1.8,5.46,"[3.55, 3.47]"
6,nuscenes,none,[],4.21,"[0.36, 6.65]"
7,nuscenes,resnet,5.87,26.5,"[9.42, 43.81]"
8,nuscenes,resnet,0.97,NaN,[]
9,nuscenes,vit,3.75,[],"[0.96, 16.62]"


In [19]:

p = '/home/jessica/personalized_driving_toyota/'

experiments = os.listdir(p)
res = {}
res_ckpt = {}
for filename in experiments: 
    if filename.endswith(".csv"):
        df = pd.read_csv(p + filename)
        df.columns = ['preds', 'targets']
        m = (df['targets'] == 0).astype(bool) | (df['targets'] > 40).astype(bool)  if "angle" not in elem  else (df['targets'] == np.inf).astype(bool)
        
        loss3 = mae_loss(torch.tensor(df['preds']),  torch.tensor(df['targets']), torch.tensor(m))
        res[filename] = round(loss3.item(), 2)
rows = []
for elem in res.keys():
    splitted = elem.split("_")
    print(elem)
    task = "multi" if 'multi' in elem else ("angle" if 'angle' in elem else 'distance')
    if "multi" in task:
        task = task + ("angle" if "angle" in elem else "distance")
    backbone = "clip" if "clip" in elem else ("resnet" if "resnet" in elem else ("none" if 'none' in elem else "vit"))
    data = 'nuscenes' if 'nuscenes' in elem else "comma"
    rows.append({"Task": task, "Backbone": backbone, "Dataset": data, "t-RMSE": round(res[elem], 2)})

angle_multi_nuscenes_multitask_none_True.csv
dist_multi_nuscenes_multitask_clip_None.csv
nuscenes_angle_vit_None.csv
nuscenes_angle_clip_None.csv
angle_multi_nuscenes_multitask_resnet_None.csv
angle_multi_nuscenes_multitask_clip_None.csv
dist_multi_nuscenes_multitask_vit_None.csv
angle_multi_nuscenes_multitask_resnet_True.csv
dist_multi_nuscenes_multitask_resnet_True.csv
nuscenes_angle_resnet_None.csv
dist_multi_nuscenes_multitask_none_True.csv
dist_multi_nuscenes_multitask_resnet_None.csv
angle_multi_nuscenes_multitask_vit_None.csv
nuscenes_distance_clip_None.csv
nuscenes_distance_none_True.csv
nuscenes_distance_resnet_None.csv


In [20]:
# The list of dictionaries
df = pd.DataFrame(rows)
data = rows

# Convert the list to a DataFrame
df = pd.DataFrame(data)

# Pivot the DataFrame
df_pivot = pd.pivot_table(df, values='t-RMSE', index=['Dataset', 'Backbone'], columns='Task', aggfunc=lambda x: x)

# Rename the columns
df_pivot.columns = [f'{col}-MAE' for col in df_pivot.columns]
# Reset the index
df_pivot = df_pivot.reset_index()
#df_pivot['Feat. Size'] = [512, 643, 512, 768, 512, 643, 512, 768]
#df_pivot = df_pivot[["Dataset", "Backbone", 'Feat. Size', 'angle-MAE', 'distance-MAE', 'multiangle-MAE', 'multidistance-MAE']].round(2)



In [21]:

df_pivot = df_pivot[["Dataset", "Backbone", 'angle-MAE', 'distance-MAE', 'multiangle-MAE', 'multidistance-MAE']].round(2)
df_pivot

,Dataset,Backbone,angle-MAE,distance-MAE,multiangle-MAE,multidistance-MAE
0,nuscenes,clip,1.18,5.57,5.01,4.17
1,nuscenes,none,NaN,5.07,4.98,6.62
2,nuscenes,resnet,0.87,25.81,"[2.88, 3.03]","[7.88, 45.1]"
3,nuscenes,vit,10.1,NaN,7.27,29.45
